**Импортируем необходимые библиотеки**

In [180]:
import cv2 as cv
import sys
import numpy as np
import math as mt
from matplotlib import pyplot as plt
import glob
import os 
np.random.seed(0)

* *Функция для сохранения картинки*

In [181]:
def SaveImage(img,name,dst=''):
    cv.imwrite(dst+name,img)

* *Функции подавлнеия шума: Билетиральный фильтр, Эрозия*

In [182]:
def ProcessBileteral(image,file_name):
    bileteral=cv.bilateralFilter(image, 20 ,  100 , 100)
    SaveImage(np.concatenate([image,bileteral],axis=1),file_name,'method/bileteral/denoising/')
    return bileteral

def ProcessErode(image,file_name,kernel):
    erode=cv.erode(image,kernel)
    SaveImage(np.concatenate([image,erode],axis=1),file_name,'method/erosion/denoising/')
    return erode

def ProcessNonLocalMeans(image,file_name):
    nonlm_img=cv.fastNlMeansDenoisingColored (image,None,10,10,7,21)
    SaveImage(np.concatenate([image,nonlm_img],axis=1),file_name,'method/non-local means/denoising/')
    return nonlm_img


In [183]:
def WatershedProcess(image,name,dst):

    hsv_image=cv.cvtColor(image,cv.COLOR_BGR2HSV)# Конвертируем изображение в HSV формат

    healty_part=cv.inRange(hsv_image,(36,10,25),(86,255,255)) # Выделяем здоровую область
    background=cv.inRange(hsv_image,(90,5,25),(240,255,255))+cv.inRange(hsv_image,(0,5,31),(2,255,255)) # Выделяем фон
    shadows=cv.inRange(hsv_image,(0,0,0),(240,255,25)) # Выделяем тень

    
    #Делаем случайную маску с шансами p1 и p2  для того чтобы доавлять маркеры случайно
    p1,p2=1,0.2
    healty_randomized_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[p1, 1-p1])  
    background_randomized_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[p2, 1-p2]) 

    #healty_mask=healty_part and healty_randomized_mask
    healty_mask=np.all([healty_part>0,healty_randomized_mask==True],axis=0)

    #background_mask=(shadows or background) and background_randomized_mask
    background_mask=np.any([background>0,shadows>0],axis=0)
    background_mask=np.all([background_mask,background_randomized_mask==True],axis=0)

    #Рисуем картинку с маркерами
    markers_visualization=np.zeros_like(image,np.uint8)
    markers_visualization[healty_mask==True]=(0,255,128)
    markers_visualization[background_mask==True]=(255,168,211)
    
    SaveImage(np.concatenate([image,markers_visualization],axis=1),name,dst+'markers/')
    

    #Делаем маркеры
    markers=np.zeros((image.shape[0],image.shape[1]),dtype='int32')

    markers[healty_mask==True]=255
    markers[background_mask==True]=1

    #Запускаем алгоритм
    labels =cv.watershed(hsv_image,markers)
    colors=[(125,0,255),(0,0,0),(125,255,0),(0,255,255)]
   
    result_img=np.zeros_like(image,np.uint8)
    i=0
    for label in np.unique(labels):
        if label == 0:
            continue
        result_img[labels == label] = colors[i]
        i+=1

    watershed_img= np.copy(result_img)
    SaveImage(np.concatenate([image,watershed_img],axis=1),name,dst+'watershed/')

    #Находим больную часть
    ill_part=labels-healty_part
    result_img[ill_part>1]=colors[3]
    SaveImage(np.concatenate([image,result_img],axis=1),name,dst+'recognizing/')

    #Сохраняем и показываем все этапы картинки
    conc=np.concatenate([image,markers_visualization,watershed_img,result_img],axis=1)
    SaveImage(conc,name,dst+'summary/')
    cv.imshow('Display window',conc)
    cv.waitKey(0)





* *Полная обработка всех изображений для разных методов обработки шума*

In [184]:
def ProcessImagesBileteral(images_paths):
    for image_path in images_paths:
        name=image_path[image_path.rfind('\\')+1:]
        image=cv.imread(image_path)
        
        bileteral_img=ProcessBileteral(image,name)
        WatershedProcess(bileteral_img,name,'method/bileteral/h=20,sigcolor=100,sigcoord=100,markers 1,0.2')
        print('\n\n')

def ProcessImagesErode(images_paths):
    for image_path in images_paths:
        name=image_path[image_path.rfind('\\')+1:]
        image=cv.imread(image_path) 
        kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
        erode_img=ProcessErode(image,name,kernel)
        WatershedProcess(erode_img,name,'method/erosion/')

def ProcessImagesNonLocalMeans(images_paths):
    for image_path in images_paths:
        name=image_path[image_path.rfind('\\')+1:]
        image=cv.imread(image_path)

        nonlm_img=ProcessNonLocalMeans(image,name)
        WatershedProcess(nonlm_img,name,'method/non-local means/')

* *Получение путей всех картинок и запуск их обработки*

In [185]:
images_paths=[path for path in glob.glob('test/*.jpg')]
ProcessImagesBileteral(images_paths)
#ProcessImagesErode(images_paths)
#ProcessImagesNonLocalMeans(images_paths)
cv.destroyAllWindows()